In [ ]:
from llm import LLMModel

def 
model = LLMModel(
    provider="openai",
    model_name="gpt-4o-2024-08-06",
    temperature=0
)

model.invoke("""

""")


In [1]:
import dspy
from typing import Dict, Any, List
from dataclasses import dataclass
from datetime import datetime
import json
import os
import pathlib
from rich.console import Console
from rich.table import Table

# Configure LLM
lm = dspy.OpenAI(model='gpt-4-turbo-preview', max_tokens=1000)
dspy.settings.configure(lm=lm)

# Set up logging directory
LOG_DIR = pathlib.Path("summarizer_logs")
PROMPT_DIR = LOG_DIR / "prompts"
os.makedirs(LOG_DIR, exist_ok=True)
os.makedirs(PROMPT_DIR, exist_ok=True)

console = Console()

# [Previous code remains the same until train_summarizer function]

def save_prompt(prompt_data: Dict, trial_num: int, predictor_num: int):
    """Save individual prompt data to file."""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = PROMPT_DIR / f"trial_{trial_num}_predictor_{predictor_num}_{timestamp}.json"
    
    with open(filename, 'w') as f:
        json.dump(prompt_data, f, indent=2)

def train_summarizer(trainset, valset=None):
    """Train the summarizer using MIPRO optimization with detailed logging."""
    
    def metric(example, pred, trace=None):
        """Metric function for evaluating summaries."""
        if not pred.summary or not pred.evaluation:
            return 0
        return pred.evaluation.overall

    # Initialize base model
    program = CodeSummarizer(max_attempts=2, num_candidates=2)
    
    # Create logging table
    progress_table = Table(title="Training Progress")
    progress_table.add_column("Trial")
    progress_table.add_column("Score")
    progress_table.add_column("Status")
    
    # Configure optimizer with logging
    teleprompter = dspy.teleprompt.MIPROv2(
        metric=metric,
        num_candidates=5,
        init_temperature=0.5,
        verbose=True
    )
    
    # Save initial configuration
    config_log = {
        "timestamp": datetime.now().isoformat(),
        "max_attempts": program.max_attempts,
        "num_candidates": program.num_candidates,
        "mipro_candidates": 5,
        "temperature": 0.5,
        "num_batches": 20
    }
    
    with open(LOG_DIR / "config.json", 'w') as f:
        json.dump(config_log, f, indent=2)
    
    best_score = 0
    best_prompts = None
    
    def log_trial(trial_num, trial_data):
        nonlocal best_score, best_prompts
        
        score = trial_data['score']
        is_pruned = trial_data.get('pruned', False)
        status = "PRUNED" if is_pruned else "COMPLETED"
        
        progress_table.add_row(
            str(trial_num),
            f"{score:.3f}",
            status
        )
        
        # Save detailed trial data
        trial_log = {
            "trial_num": trial_num,
            "timestamp": datetime.now().isoformat(),
            "score": score,
            "status": status,
            "predictors": []
        }
        
        # Log each predictor's prompts
        if not is_pruned:
            for i, predictor in enumerate(trial_data['program'].predictors()):
                predictor_data = {
                    "predictor_num": i,
                    "type": predictor.__class__.__name__
                }
                
                if hasattr(predictor, 'extended_signature'):
                    predictor_data["instructions"] = predictor.extended_signature.instructions
                elif hasattr(predictor, 'signature'):
                    predictor_data["instructions"] = predictor.signature.instructions
                
                trial_log["predictors"].append(predictor_data)
                save_prompt(predictor_data, trial_num, i)
        
        # Save trial log
        with open(LOG_DIR / f"trial_{trial_num}.json", 'w') as f:
            json.dump(trial_log, f, indent=2)
        
        # Update best score and prompts
        if score > best_score and not is_pruned:
            best_score = score
            best_prompts = trial_log
            with open(LOG_DIR / "best_prompts.json", 'w') as f:
                json.dump(best_prompts, f, indent=2)
    
    # Compile with progress tracking
    optimized_program = teleprompter.compile(
        program,
        trainset=trainset,
        valset=valset,
        max_bootstrapped_demos=2,
        max_labeled_demos=2,
        num_batches=20
    )
    
    # Log all trials
    for trial_num, trial_data in optimized_program.trial_logs.items():
        log_trial(trial_num, trial_data)
    
    # Display final progress table
    console.print(progress_table)
    
    # Save final summary
    summary = {
        "total_trials": len(optimized_program.trial_logs),
        "best_score": best_score,
        "final_score": list(optimized_program.trial_logs.values())[-1]['score'],
        "timestamp": datetime.now().isoformat()
    }
    
    with open(LOG_DIR / "training_summary.json", 'w') as f:
        json.dump(summary, f, indent=2)
    
    return optimized_program

def main():
    # [Previous main() code remains the same until training]
    
    # Train and test optimized model with logging
    print("\nTraining optimized model...")
    console.print("[bold blue]Starting training with detailed logging...[/bold blue]")
    console.print(f"Logs will be saved to: {LOG_DIR}")
    
    optimized_summarizer = train_summarizer(trainset)
    
    # Load and display best prompts
    with open(LOG_DIR / "best_prompts.json", 'r') as f:
        best_prompts = json.load(f)
    
    console.print("\n[bold green]Best Performing Prompts:[/bold green]")
    for predictor in best_prompts["predictors"]:
        console.print(f"\nPredictor {predictor['predictor_num']}:")
        console.print(predictor['instructions'])
    
    # [Rest of the main() function remains the same]

if __name__ == "__main__":
    main()

 		You are using the client GPT3, which will be removed in DSPy 2.6.
 		Changing the client is straightforward and will let you use new features (Adapters) that improve the consistency of LM outputs, especially when using chat LMs. 

 		Learn more about the changes and how to migrate at
 		https://github.com/stanfordnlp/dspy/blob/main/examples/migration.ipynb


Generated Summary: The code defines a function in Python that calculates the nth Fibonacci number using a loop.

Evaluation Scores:
Accuracy: /10
Completeness: 9

Completeness Score: 8/10
Conciseness: 7/10

Rationale: Accuracy Score: 9
The summary accurately describes the purpose of the code and how it calculates the nth Fibonacci number using a loop. However, it could be improved by mentioning that the function returns the Fibonacci number.

Completeness Score: 8
The summary provides a good overview of what the code does, but it could be more detailed by explaining the logic behind the Fibonacci sequence and how the code achieves the calculation.

Conciseness Score: 7
The summary is concise and to the point, but it could be improved by including more specific details about the code implementation and how it works.
